#### Import libraries

In [1]:
from utilities.logger import Logger
from utilities.utilities import *
import datetime
from tournaments.tournaments_wta import *
from matches.match_details import *
from matches.match import *
import time
import os
import functools
from typing import *
import pprint
import unidecode
import glob

ModuleNotFoundError: No module named 'utilities'

In [ ]:
from typing import *
def string_normalize(string):
    return unidecode.unidecode(string).replace(" ", "_")

def identify_missing_tournaments(year:str) -> List[str]:
    """
    for a given years return missings tournaments
    Args
        - year => must be a valid string 
    Return
        - list of tournaments names if we missed some
    Example
        >>>identify_missing_tournaments(2018)
            ['tourney_a', 'tourney_b']
    """
    assert year.isdigit(), "annee non numerique"
    try:
        tournaments_for_the_year_values = get_tournaments_for_year(year)[1]
        tournaments_for_the_year_values = tournaments_for_the_year_values.drop_duplicates() #eliminate duplicates
        tournaments_for_the_year_values['title_modified'] = tournaments_for_the_year_values["title"].apply(lambda x: string_normalize(x))
        print(f"nbre tournois before cut:{len(tournaments_for_the_year_values)}")
        #we have now in the data ITF, without much information on the data
        tournaments_for_the_year_values = tournaments_for_the_year_values.query("not title.str.contains('ITF')")
        
        print(f"nbre tournois:{len(tournaments_for_the_year_values)}")
                                    
    except Exception as e:
        print(f"impossible de recuperer les matches pour l'annee {year}")
    
    existing_tournament = list(os.listdir(os.path.join(os.getcwd(), 'prd-trd', '0_raw', year)))
    print(f"existing tournaments list: {len(existing_tournament)}")
    #print(f"existing tourney {year}: {set(tournaments_for_the_year_values['title_modified'].to_list())}")
    difference = list(set(tournaments_for_the_year_values['title_modified'].to_list()).difference(set(existing_tournament)))
    #print(f"difference {year}: {difference}")

    #print(f"tournoi deja recuperes pour annee {year}: {len(existing_tournament)}")
    return tournaments_for_the_year_values.query("title_modified in @difference").drop(["title_modified"], axis=1)


In [ ]:
import re
from datetime import datetime

def parsing_tournament_date(date: str) -> tuple:
    """
    Take the date of the tournament and parse it to return:
    - Start date of the tournament
    - End date of the tournament 
    
    Args
    --------
    date : str
        The date to parse, e.g., ' Jul 3   Jul 16, 2023 '
        
    Returns
    -------
    tuple
        (date_debut, date_fin) : (03/07/2023, 16/07/2023)

    """
    
    # Normalize spaces to single spaces and trim the string
    normalized_date = ' '.join(date.strip().split())

    # Extract month-day pairs using regex
    dates = re.findall(r"\b\w{3} \d+", normalized_date)
    if len(dates) != 2:
        raise ValueError("Invalid date format!")
    
    # Extract the year from the normalized date string
    year = re.search(r"\d{4}", normalized_date)
    if not year:
        raise ValueError("Year not found in the date!")
    year = year.group(0)

    # Append the year to each date fragment
    dates_with_year = [f"{elt} {year}" for elt in dates]

    # Convert the date fragments to datetime format
    date_debut = datetime.strptime(dates_with_year[0], "%b %d %Y")
    date_fin = datetime.strptime(dates_with_year[1], "%b %d %Y")

    # Convert each datetime object to the desired string format
    """
    date_debut_str = date_debut.strftime("%d/%m/%Y")
    date_fin_str = date_fin.strftime("%d/%m/%Y")
    """
    
    return date_debut, date_fin



In [ ]:
year = "2023"
missing_tournaments = identify_missing_tournaments(year)

In [ ]:
missing_tournaments

In [ ]:
date_today = datetime.now().date()

date_parsing = missing_tournaments["dates"].apply(parsing_tournament_date)
missing_tournaments["date_debut"] = date_parsing.apply(lambda x:x[0])
missing_tournaments["date_fin"] = date_parsing.apply(lambda x:x[1])

missing_tournaments = missing_tournaments.query("date_fin<=@date_today")
missing_tournaments